# Lab: tf.layers

`tf.layers` を使うと行列演算や Variable の存在を隠蔽しつつ、柔軟にニューラルネットを記述することができます。
TensorFlow v1.0 で contrib から外れて、変更が加わりにくい安定したモジュールになりました。
楽さと柔軟さのバランスも取れており、おすすめの書き方です。

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

print(tf.__version__)

In [ ]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
x_ph = tf.placeholder(tf.float32, [None, 784])
y_ph = tf.placeholder(tf.float32, [None, 10])

In [ ]:
hidden = tf.layers.dense(x_ph, 20)
logits = tf.layers.dense(hidden, 10)
y = tf.nn.softmax(logits)

In [ ]:
cross_entropy = -tf.reduce_mean(y_ph * tf.log(y))

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y_ph, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
train_op = tf.train.GradientDescentOptimizer(1e-1).minimize(cross_entropy)

In [ ]:
init_op = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init_op)
    for i in range(3001):
        x_train, y_train = mnist.train.next_batch(100)
        sess.run(train_op, feed_dict={x_ph: x_train, y_ph: y_train})
        if i % 100 == 0:
            train_loss = sess.run(cross_entropy, feed_dict={x_ph: x_train, y_ph: y_train})
            test_loss = sess.run(cross_entropy, feed_dict={x_ph: mnist.test.images, y_ph: mnist.test.labels})
            tf.logging.info("Iteration: {0} Training Loss: {1} Test Loss: {2}".format(i, train_loss, test_loss))
    test_accuracy = sess.run(accuracy, feed_dict={x_ph: mnist.test.images, y_ph: mnist.test.labels})
    tf.logging.info("Accuracy: {}".format(test_accuracy))